In [1]:
import mne
import numpy as np
import scipy
from matplotlib import pyplot as plt
from filterpy.kalman import KalmanFilter
import pandas as pd

import sys
sys.path.insert(1, r'C:\Users\ICN_admin\Documents\icn\icn_m1\restructure')

import features_df

In [2]:
PATH_ = r"C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS_Berlin\sub-002\ses-20200131\ieeg\sub-002_ses-20200131_task-SelfpacedRotationR_acq-MedOn+StimOff_run-4_ieeg.vhdr"
rawarr = mne.io.read_raw_brainvision(PATH_) #np.random.random([1000, 7])

Extracting parameters from C:\Users\ICN_admin\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\Datasets\BIDS_Berlin\sub-002\ses-20200131\ieeg\sub-002_ses-20200131_task-SelfpacedRotationR_acq-MedOn+StimOff_run-4_ieeg.vhdr...
Setting channel info structure...


In [3]:
ch_names = ["ECOG_RIGHT_0", "ECOG_RIGHT_1", "ECOG_RIGHT_2", "ECOG_RIGHT_3", "ECOG_RIGHT_4",\
           "ECOG_RIGHT_5", "ECOG_RIGHT_6"]

settings_features = {
    "FrequencyRanges": [[4, 8], [8, 12], [13, 20], [20, 35], [13, 35], [60, 80], [90, 200], [60, 200], [200, 300]],
    "FeatureLabels": ["theta", "alpha", "low beta", "high beta", "all beta", "low gamma", "HFA", "all gamma", "high HFA"],
    "Seglengths": [1, 2, 2, 3, 3, 3, 10, 10, 10, 10], 
    "BandpowerMethod" : {
        "BandpassFilter" : True, 
        "Wavelet" : False,
    },
    "BandpowerFeatures" : {
        "Activity" : True, 
        "Mobility" : True,
        "Complexity" : True, 
    },
    "RawHjorthParams" : True,
    "SharpwaveFilter": {
        "bp_low_cutoff" : 5, 
        "bp_high_cutoff" : 90
    },
    "Sharpwave" : {
        "MaxTroughProminence" : True,
        "MaxPeakProminence" : True,
        "MaxTroughSharpness" : True, 
        "MaxPeakSharpness" : True, 
        "MeanTroughSharpness" : True,
        "MeanPeakSharpness" : True,
    }, 
    "KalmanfilterParams" : {
        "Tp" : 0.1, 
        "sigma_w" : 0.7, 
        "sigma_v" : 1
    },
    "KalmanfilterFrequencybands" : [False,False,False,True,True,True,True,True,True]
}

In [4]:
f_object = features_df.features(settings_features, fs=1375, line_noise=50, channels=ch_names)

No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 2.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 10.00 Hz)
- Filter length: 1377 samples (1.001 sec)

No data specified. Sanity checks related to the length of the signal relative to the filter order will be skipped.
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB s

In [10]:
pd_features = pd.DataFrame()

In [11]:
data_batch = rawarr.get_data()[:7,:1375]

In [13]:
pd_features = pd_features.append(f_object.estimate_features(data_batch), ignore_index=True)

In [14]:
data_batch = rawarr.get_data()[:7,1375:1375*2]

In [15]:
pd_features = pd_features.append(f_object.estimate_features(data_batch), ignore_index=True)

In [16]:
pd_features

,ECOG_RIGHT_0_BandpassFilter_Activity_HFA,ECOG_RIGHT_0_BandpassFilter_Activity_all beta,ECOG_RIGHT_0_BandpassFilter_Activity_all gamma,ECOG_RIGHT_0_BandpassFilter_Activity_alpha,ECOG_RIGHT_0_BandpassFilter_Activity_high HFA,ECOG_RIGHT_0_BandpassFilter_Activity_high beta,ECOG_RIGHT_0_BandpassFilter_Activity_low beta,ECOG_RIGHT_0_BandpassFilter_Activity_low gamma,ECOG_RIGHT_0_BandpassFilter_Activity_theta,ECOG_RIGHT_0_BandpassFilter_Complexity_HFA,...,ECOG_RIGHT_6_BandpassFilter_Mobility_theta,ECOG_RIGHT_6_RawHjorth_Activity,ECOG_RIGHT_6_RawHjorth_Complexity,ECOG_RIGHT_6_RawHjorth_Mobility,ECOG_RIGHT_6_Sharpwave_MaxPeakSharpness,ECOG_RIGHT_6_Sharpwave_MaxPeakprominence,ECOG_RIGHT_6_Sharpwave_MaxTroughProminence,ECOG_RIGHT_6_Sharpwave_MaxTroughSharpness,ECOG_RIGHT_6_Sharpwave_MeanPeakSharpness,ECOG_RIGHT_6_Sharpwave_MeanTroughSharpness
0,2.680569,11.984599,3.843715,77.799385,2.296347,7.545327,14.385181,1.689331,170.433789,0.678181,...,0.029495,558.963829,6.512436,0.181290,-0.152229,86.273028,86.273028,-0.152229,-1.466366,-1.466366
1,2.767075,11.995733,3.986817,15.912057,2.344093,7.487756,13.104710,1.830290,63.935867,0.756457,...,0.027457,670.901552,8.059607,0.150718,-0.379691,36.748575,36.748575,-0.379691,-1.404599,-1.404599


In [19]:
pd_features.iloc[0]

ECOG_RIGHT_0_BandpassFilter_Activity_HFA           2.680569
ECOG_RIGHT_0_BandpassFilter_Activity_all beta     11.984599
ECOG_RIGHT_0_BandpassFilter_Activity_all gamma     3.843715
ECOG_RIGHT_0_BandpassFilter_Activity_alpha        77.799385
ECOG_RIGHT_0_BandpassFilter_Activity_high HFA      2.296347
                                                    ...    
ECOG_RIGHT_6_Sharpwave_MaxPeakprominence          86.273028
ECOG_RIGHT_6_Sharpwave_MaxTroughProminence        86.273028
ECOG_RIGHT_6_Sharpwave_MaxTroughSharpness         -0.152229
ECOG_RIGHT_6_Sharpwave_MeanPeakSharpness          -1.466366
ECOG_RIGHT_6_Sharpwave_MeanTroughSharpness        -1.466366
Name: 0, Length: 252, dtype: float64